In [1]:
import numpy as np
import random
from timeit import default_timer as timer
from collections import deque

class Node:
    def __init__(self, id):
        self.id = id
        self.position = None
        self.neighbors = None
    
    def show_param(self):
        print(self.id, self.position, self.neighbors)

    def get_position(self):
        x = random.randint(-100, 100)
        y = random.randint(-100, 100)
        z = random.randint(0, 50)
        self.position = np.asarray([x, y, z])
    
    def get_connections(self, num_cit, possibility):
        con = []
        for i in range(num_cit):
            if i == self.id:
                pass
            else:
                a = random.randint(1, 100)
                if a <= possibility:
                    con.append(i)
        self.neighbors = con
    
num_cit = 5
cities_lst = []
for i in range(num_cit):
    node = Node(i)
    node.get_position()
    node.get_connections(5, 90)
    node.show_param()
    cities_lst.append(node)

def calculate_cost(routes, cities, assimetrial=False):
    costs = []
    window = 2
    if assimetrial == False:
        for route in routes:
            cost = 0
            for i in range(len(route)):
                if route[i] == route[-1]:
                    cost = cost + np.linalg.norm(cities[route[-1]].position - cities[route[0]].position)
                    costs.append(cost)   
                else:
                    for j in range(len(route)):
                        a = route[j:j+window]
                        if len(a) == 2:
                            cost = cost + np.linalg.norm(cities[a[0]].position - cities[a[1]].position) 
        return costs
    elif assimetrial != False:
        for route in routes:
            cost = 0
            for i in range(len(route)):
                if route[i] == route[-1]:
                    direction = cities[route[-1]].position[2] - cities[route[0]].position[2]
                    if direction > 0 or direction < cities[route[-1]].position[2]:
                        cost = cost + (np.linalg.norm(cities[route[-1]].position - cities[route[0]].position)-np.linalg.norm(cities[route[-1]].position - cities[route[0]].position)*0.1)
                        costs.append(cost)   
                    else: 
                        cost = cost + (np.linalg.norm(cities[route[-1]].position - cities[route[0]].position)+np.linalg.norm(cities[route[-1]].position - cities[route[0]].position)*0.1)
                        costs.append(cost)  
                else:
                    for j in range(len(route)):
                        a = route[j:j+window]
                        if len(a) == 2:
                            direction = cities[a[0]].position[2] - cities[a[1]].position[2]
                            if direction > 0 or direction < cities[0].position[2]:
                                cost = cost + (np.linalg.norm(cities[a[0]].position - cities[a[1]].position)-np.linalg.norm(cities[a[0]].position - cities[a[1]].position)*0.1) 
                            else:
                                cost = cost + (np.linalg.norm(cities[a[0]].position - cities[a[1]].position)+np.linalg.norm(cities[a[0]].position - cities[a[1]].position)*0.1)
        return costs

0 [-35 -89  24] [1, 2, 3, 4]
1 [-79  42  50] [0, 2, 3, 4]
2 [66 77 38] [0, 1, 3, 4]
3 [29 17  6] [1, 2, 4]
4 [-81  -8   5] [0, 3]


In [2]:
def dfs(city_map, start_point, path=None, path_lst=[]):
    if path is None:
        path = [start_point]

    if len(path) == len(city_map):
        path_lst.append(path)

    possible = city_map[start_point].neighbors
    for i in possible:
        if i not in path:
            new_path = path.copy()
            new_path.append(i)
            dfs(city_map, i, new_path, path_lst)

    return path_lst

start = timer()
dfs_paths = dfs(cities_lst, 0)
costs_dfs = calculate_cost(dfs_paths, cities_lst)
end = timer()
time_dfs = end - start


In [3]:
def bfs(graph, start):
    queue = deque([(start, [start])])
    all_paths = []

    while queue:
        current_node, path = queue.popleft()

        if len(path) == len(graph):
            all_paths.append(path)
            continue

        for neighbor in graph[current_node].neighbors:
            if neighbor not in path:
                new_path = path + [neighbor]
                queue.append((neighbor, new_path))

    return all_paths

start = timer()
bfs_paths = bfs(cities_lst, 0)
costs_bfs = calculate_cost(bfs_paths, cities_lst)
end = timer()
time_bfs = end - start

In [4]:
### Wyniki
def best_route(costs, paths):
    min_cost = min(costs)
    for cost in range(len(costs)):
        if min_cost == costs[cost]:
            indx = cost
    return(min_cost, paths[indx])

print(f'Najkrótsza scieżka dla DFS oraz jej koszt: {best_route(costs_dfs, dfs_paths)} w czasie: {time_dfs}')
print(f'Najkrótsza scieżka dla BFS oraz jej koszt: {best_route(costs_bfs, bfs_paths)} w czasie: {time_bfs}')
print(f'Najkrótsza scieżka dla BFS oraz jej koszt: {best_route(costs_bfs, bfs_paths)} w czasie: {time_bfs}')

Najkrótsza scieżka dla DFS oraz jej koszt: (1772.9984062620474, [0, 3, 2, 1, 4]) w czasie: 0.0017486000433564186
Najkrótsza scieżka dla BFS oraz jej koszt: (1772.9984062620474, [0, 3, 2, 1, 4]) w czasie: 0.001132899895310402
